# Exercise 1: Quadrotor Flight with Deep Learning
---
## Table of Contents
[Kaggle Setup](#kaggle-environment-setup)
1. [Introduction](#introduction)
2. [Setup and Configuration](#setup-and-configuration)
3. [Data Preparation](#data-preparation)
   - [Dataset](#dataset)
   - [Visualization of Dataset](#visualization-of-dataset)
   - [Data Loading](#data-loading)
4. [Model Architecture](#model-architecture)
5. [Training the Model](#training-the-model)
   - [Loss Function and Optimizer](#loss-function-and-optimizer)
   - [Training Loop](#training-loop)
   - [Saving and Loading Model ](#saving-and-loading-model)
6. [Closed-Loop Model Evaluation](#closed-loop-model-evaluation)

## Kaggle Environment Setup:

Here we will explain a basic way of setting up your this notebook in Kaggle, such that you can work on the exercises in the online environment. This is for students who have had difficulties installing the environment/dependencies, or simply having issues initializing the kernel.


### Creating the Notebook

Creating the notebook can be done by simply pressing the create button in the top left corner of the kaggle home page. From there select import notebook. This will open a window which will allow you to navigate to your local repository where you should find `this notebook` (NOT the ex_1.ipynb notebook) and then open. This will then create the notebook in kaggle for you.

### Creating the Dataset

The second part requires some additional work. Using the same create button, create a dataset. Here you will have the option to name the dataset. Give it the name `AE4353_1`. This is what the exercise file here-after will use to denote it. From there, you will need to upload files. This can be done in 2 parts:

1. Upload the datafiles `2D_QUAD_HOVER.npz` and `3D_QUAD_HOVER.npz`.
2. Upload the folder `additional` found in the ex_1 folder. These are all the plotting scripts and dataloader which are required for your code to run, but we kept seperate to keep the notebook simple and clean. Once you have added the 6 files, move onto the next step.

> MAKE SURE TO UPLOAD THE WHOLE FOLDER! YOU CAN DO THIS BY DRAGGING AND DROPPING IT INTO THE UPLOAD BOX!

If there are any confusions feel free to contact any of the TA's, or come down during the exercise session, we would be happy to help you out!

### Connecting the two:

From your notebook in the top left cotner there is a `input` section. There you can press the `+ add input` button, and add the dataset you just created. This way Kaggle knows you wish to use this dataset for the notebook. Once this is done, you are ready to go!

## Introduction
Welcome to this exercise, where you will build and train a neural network using PyTorch to control a quadrotor drone. Throughout this process, you will explore the fundamentals of a deep learning pipeline, including data preparation, model architecture, training, and validation. 

Your objective is to train a model that can take in the state information of a quadrotor drone—such as its position, velocity, attitude, and angular velocity—and output the control signals needed (e.g., motor thrust) to guide the drone to the origin (X = Y = Z = 0) as quickly and efficiently as possible (balancing the trade-off between minimizing time and energy consumption).

To assist you, we’ve already generated a dataset of time-optimal trajectories in a simulation. These trajectories were computed using [Model Predictive Control (MPC)](https://www.do-mpc.com/en/latest/theory_mpc.html) in a simulated environment. Calculating such time-optimal trajectories is computationally intensive, but if we can approximate them using a neural network, we can make real-time control feasible even on platforms with limited computational resources, such as small drones. Let’s get started and explore how we can achieve this!

## Setup and Configuration
---
Machine learning programs consist of many components—datasets, dataloaders, models, and optimizers—each requiring numerous parameters known as hyperparameters for optimal performance. Instead of hardcoding these parameters directly into your code, it is considered best practice to seperate the configuration information from the code. Although this is commonly done in a seperate "config" file, here we simplify it and do it in its own "code block". In General, seperating your configuration information provides several advantages:

- **Flexibility**: Modify settings without altering the core code.
- **Error Reduction**: Reduce the risk of errors when adjusting parameters.
- **Consistency**: Ensure uniform settings across different experiments and runs.
- **Portability**: Easily transfer code between different environments, such as from a local machine to a cloud platform for large-scale training.

Please execute the below cell to load the hyperparameter configuration into Python.

In [ ]:
trajectory_dim = 2
dataset_file = "/kaggle/input/ae4353-1/2D_QUAD_HOVER.npz" #Adjust this path to your dataset location

batchsize_train = 1024
batchsize_val = 4096
shuffle_train = True
shuffle_val = False
num_workers_train_loader = 0
num_workers_val_loader = 0

model_hidden_dims = 120
model_num_layers = 4

epochs = 2
validate_per_steps = 500

## Data Preparation
---
Proper data preparation is essential for building effective machine learning models. This section covers three crucial tasks:

1. **Dataset**: Defining and implementing how data is organized and accessed using PyTorch’s dataset classes.
2. **Visualization of Dataset**: Using tools like [Rerun](https://rerun.io/) to explore and understand the data through visualization.
3. **Data Loading**: Efficiently batching and loading data using PyTorch’s `DataLoader` to streamline the training process.

These steps ensure your data is ready for training and validation.

### Dataset
In PyTorch, datasets can be loaded using two types of dataset objects: map-style datasets (`torch.utils.data.Dataset`) and iterable-style datasets (`torch.utils.data.IterableDataset`).

- **Map-style datasets**: These use an index to access data samples. As a rule of thumb, these should be used if the entire dataset is available from the start. Therefore, this is the type we will use in this exercise.
- **Iterable-style datasets**: These are used for data that is generated or streamed in real-time, such as live sensor data.

To work with data in PyTorch, you often need to define a custom dataset class. For map-style datasets, this involves subclassing the abstract `torch.utils.data.Dataset` class and implementing two methods:

- **`__len__()`**: Returns the total number of data samples in the dataset.
- **`__getitem__(index)`**: Retrieves the data sample corresponding to the provided index.

A sample implementation is as follows:

- **`TrajectoryDataset` Class**: This class handles our trajectory dataset.
  - **`flatten=False`**: Each sample is a full trajectory with multiple timesteps.
  - **`flatten=True`**: Each sample represents a single timestep within a trajectory. This setup helps achieve a more i.i.d. (independent and identically distributed) dataset by shuffling the order of input as much as possible, which is beneficial during training. Therefore, we set `flatten=True` for training.

- **`prepare_dataset` Function**: This function loads the dataset stored in numpy format, using the first 80% of the trajectories as the training set and the remaining 20% as the validation set.

Please execute the following cell to utilize the dataset preparation features. This will load the `TrajectoryDataset` class and the `prepare_dataset` function.

In [ ]:
import torch
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
import numpy as np

import shutil
import os

shutil.copytree('kaggle/input/AE4353_1/additional', '/kaggle/working/additional')

from additional.dataloader import TrajectoryDataset, prepare_dataset

Now let’s get the dataset ready for training! Run the cell below, and you will get two PyTorch datasets back: one for training (with flattened trajectories) and one for validation (with non-flattened trajectories).

**Note**: Here we provide an extract of code to show you how to load the parameters from the YAML file. If you intend to use this method, please make sure to try it out throughout the exercise, as we will not provide further examples on how to "load" the parameters in this way.

In [ ]:
train_set, val_set = prepare_dataset(dataset_file, trajectory_dim)

### Data Loading
In PyTorch, the `DataLoader` is a utility that handles the loading and batching of data from a dataset during training. When working with a map-style dataset, the `DataLoader` is responsible for efficiently fetching and organizing data into batches for the training process.

Key arguments that enhance the functionality of the `DataLoader` include:

- **`batch_size`**: This argument specifies the number of samples per batch. For example, if `batch_size=32`, the `DataLoader` will group 32 samples together and return them as a single batch during each iteration.

- **`shuffle`**: When set to `True`, the `DataLoader` shuffles the data at the beginning of each epoch. This helps prevent the model from learning the order of the data, which improves generalization and reduces overfitting.

- **`num_workers`**: This argument defines the number of subprocesses to use for data loading. A higher value can speed up data loading by parallelizing the process. A value of 0 means data loading will occur in the main thread (i.e. no additional dataloader processes are launched). This is useful for debugging, as it avoids the complexity of dealing with concurrent processes. For small datasets, a value of 0 is often sufficient.

In practice, the `DataLoader` manages data fetching and batching efficiently, allowing the training loop to focus on processing the data rather than handling the complexities of data loading.


Execute the cell below to create training and validation set loaders.

In [ ]:
from torch.utils.data import DataLoader


# Create training set loader
train_loader = DataLoader(
    train_set,
    batch_size=batchsize_train,
    shuffle=shuffle_train,
    num_workers=num_workers_train_loader,
)

# Create validation set loader
val_loader = DataLoader(
    val_set,
    batch_size=batchsize_val,
    shuffle=shuffle_val,
    num_workers=num_workers_val_loader,
)

## Model Architecture
---
In PyTorch, neural networks are built using reusable modules such as layers, activation functions, and various operations. These modules can be combined and organized to create complex neural networks. To define a custom module, you subclass the `torch.nn.Module` class and implement the `forward` method. The `forward` method specifies how input data is transformed as it passes through the network.

The `nn.Module` class offers a flexible way to create both individual layers and entire networks. `nn.Module` objects can be nested within each other, facilitating the construction of sophisticated models. For example, you can create a custom layer by subclassing `nn.Module`, like `MyCustomLayer(nn.Module)`. This layer can then be used within another `nn.Module` object, such as `MyCustomNetwork(nn.Module)`. You instantiate the custom layer in `MyCustomNetwork`'s `__init__` method (e.g., `self.custom_layer = MyCustomLayer()`) and call it in the `forward` method (e.g., `self.custom_layer(...)`). This modular approach enhances code reuse and clarity.

PyTorch also provides the `nn.Sequential` container, which is a subclass of `nn.Module`. It allows for the sequential chaining of modules, where the output of one module’s forward method is automatically fed as the input to the next module. This makes it easy to build simple feedforward networks without manually defining the data flow between layers.

### nn.Parameter
In PyTorch, `nn.Parameter` is a special type of tensor designed to be used as a parameter within an `nn.Module`, such as the weights and biases in a `nn.Linear` layer.

When a tensor is wrapped in an `nn.Parameter` and assigned  **as an attribute** of an `nn.Module`, it is automatically included in the module’s list of parameters. This ensures it will be returned by the module’s `parameters()` method, which is essential for optimization. Regular tensors do not get automatically registered as parameters, even if they are used within an `nn.Module`.

By default, `nn.Parameter` tensors have `requires_grad=True`, meaning gradients will be computed for these tensors during backpropagation. In contrast, regular tensors default to `requires_grad=False`, unless explicitly set otherwise.

Parameters of `nn.Module` submodules that are registered as attributes are also recognized as parameters of the model. This makes it easy to track trainable parameters, as calling `model.parameters()` includes both the model's and its submodules' parameters. Additionally, model saving and loading utilities (`torch.save` and `torch.load`) handle the entire model's parameters, including those of its submodules.

### Model for This Exercise
In this exercise, we use a Multi-Layer Perceptron (MLP) network to map input to output. This network consists of 4 hidden layers of size 120, each connected by linear layers. To introduce non-linearity, we apply the Rectified Linear Unit (ReLU) activation function to the hidden units in each layer.

PyTorch provides implementations for commonly used layer types and activation functions, such as linear layers (`nn.Linear`), 2D convolutional layers (`nn.Conv2d`), ReLU (`nn.ReLU`), and Sigmoid (`nn.Sigmoid`).

`optional`: before the first linear layer, you can normalize the input to a normal distribution with zero mean and unit variance ($
\mathcal{N}(0, 1)$). This normalization ensures that all input features are on a similar scale, which can accelerate convergence during training. It helps prevent certain features from dominating the learning process due to scale differences and maintains stable gradients throughout the network. This however is not a requirement!


In [ ]:
from torch import nn

# -------------------------------------------
#  TODO: Define the model architecture
# -------------------------------------------
model = None
# -------------------------------------------
# END
# -------------------------------------------

if model is None:
    raise ValueError("Model architecture is not defined. Please implement the model.")

print(model)

## Training the Model
---
Training a machine learning model involves optimizing its parameters to minimize some relationship between its predictions and the input data. This process requires defining a loss function that quantifies how well the model performs and choosing an optimizer to adjust the model’s parameters based on the computed gradients.

### Loss Function and Optimizer
To train a model effectively, we need a loss function that quantifies how well the model fits the data. In this exercise, we use the Mean Squared Error (MSE) loss, which measures the discrepancy between the model’s predicted control actions and the ground truth optimal actions.

Alongside the loss function, we require an optimizer to adjust the model's parameters. When instantiating the optimizer, we need to pass in the set of parameters to be learned. This allows the optimizer to track the gradients of those parameters and maintain statistics that can help the model converge more quickly and robustly. 

In our case, we adopt the widely-used Adam optimizer. It comes with several hyperparameters, with the learning rate being the most crucial. We will configure the learning rate while keeping the other hyperparameters at their default values.


Execute the cell below to set the loss function and optimizer.

In [ ]:
criterion = torch.nn.MSELoss()  # Define the Mean Squared Error loss function

# Optimizer
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Use the Adam optimizer to update the model parameters

### Training Loop
A typical training loop consists of an outer loop that iterates over epochs (where one epoch is a full pass through the entire training set) and an inner loop where the data loader iterates over batches of data within each epoch.

While it is not strictly necessary to set the model to training mode using `model.train()`, it is good practice to do so. This is because certain `nn.Module` components, such as [dropout layers](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html) and [batch normalization layers](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html), behave differently depending on whether the model is in training or evaluation mode.

The main steps of the training loop are as follows:
- **`optimizer.zero_grad()`**: This step clears the gradients of all parameters. In PyTorch, gradients accumulate by default, so it’s essential to zero them out at the start of each iteration to prevent mixing gradients from different batches.
- **`outputs = model(input)`**: The model performs a forward pass with the input data, generating predictions based on the current state of its parameters.
- **`loss = criterion(outputs, targets)`**: The loss function (criterion) compares the model’s predictions to the true target values and computes the loss, a scalar that indicates how well the model’s predictions match the targets. This value is used to guide the model's learning.
- **`loss.backward()`**: This step calculates the gradients of the loss with respect to each model parameter using backpropagation. PyTorch’s autograd engine performs this computation, determining how to adjust each parameter to minimize the loss.
- **`optimizer.step()`**: The optimizer updates the model’s parameters based on the computed gradients. Using the Adam algorithm, the optimizer adjusts the parameters in the direction that reduces the loss, improving the model’s performance over time.


Ready to see our model in action? Run the cell below to start the training loop, which includes a validation step every 500 steps. During these validations, we’ll generate plots to highlight the model’s worst predictions and evaluate its performance at various loss percentiles. Additionally, the validation loss will be logged. Let’s dive in and get started with training!

Explain the argument class setup (in the def)


<strong style="color:red;"> 1.2: TODO: Start the training and validation loop</strong>


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt


def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    validate_every: int = 1,
):
    train_losses = []
    val_losses = []

    # -------------------------------------------
    #  TODO: Define the training loop
    # -------------------------------------------
    pass
    # --------------------------------------------
    # END
    # --------------------------------------------

    return train_losses, val_losses

def validate(model: nn.Module, val_loader: DataLoader) -> float:
    model.eval()git
    total_loss = 0.0

    # --------------------------------------------
    # TODO Define the validation loop
    # --------------------------------------------
    pass
    # --------------------------------------------
    # END
    # --------------------------------------------

    return total_loss / len(val_loader)


<strong style="color:red;"> 1.3 TODO: Complete the training loop, and tune the learning rate paramater to achieve fast and effective learning.</strong>

You can expect a validation error around 0.003 after 2 epochs of training.

In [ ]:
# ---------------------------------------------
# TODO: Complete training loop
# ---------------------------------------------
train_losses, val_losses = None, None

# ---------------------------------------------
# END
# ---------------------------------------------

if train_losses is None or val_losses is None:
    raise ValueError("Training loop did not complete successfully. Please implement the training loop.")

In [ ]:
# Plotting the training and validation losses

from additional.plot_utils import plot_multiple_sample_predictions

plot_multiple_sample_predictions(model, val_set, num_samples=2)

### Saving and Loading the Model
After training the model, it’s important to save its weights so you can reuse the model later. The best practice is to save only the model parameters using `model.state_dict()`, rather than saving the entire model. This approach avoids the security risks associated with serializing the Python code that defines the model architecture, which could potentially expose the system to malicious code when reloaded.

To load a saved model, first instantiate a new model object and then use the `load_state_dict()` method to load the saved parameters into this model object.


Run the cell below to save the trained model and then reload it.

In [ ]:
# Save model
torch.save(model.state_dict(), "model_trained.pt")

# Load model
model.load_state_dict(torch.load("model_trained.pt", weights_only=True))

## Closed-Loop Model Evaluation
---
After training the model, it is essential to evaluate its performance to ensure it operates as intended in closed-loop.

In this section, we will deploy the trained model in closed-loop simulation to evaluate its interaction with the system it is designed to control. Below, we have set up a controller based on the trained model, defined the system dynamics of a 2D quadrotor drone, and prepared a simulation environment to evaluate the trained model.

<strong style="color:red;">TODO 1.4: Implement the neural-network-based controller. Pay attention to how the model should be called differently now versus training time.</strong>

Then run the cell to establish the system dynamics, and set up the simulation environment for evaluating the model.

In [ ]:
# Define controllergit remote -v
def controller(x: np.ndarray) -> np.ndarray:
    """
    Compute the control output using the trained model.

    Parameters:
        x (np.ndarray): The state vector.

    Returns:
        np.ndarray: Control output as a numpy array.
    """
    # -----------------------------------------------
    # TODO: Implement the controller function
    # -----------------------------------------------
    pass
    # -----------------------------------------------
    # END
    # -----------------------------------------------

Now, run the cell below to evaluate the effectiveness of the trained model by simulating the quadrotor's trajectories controlled by the model. Try experimenting with different initial conditions (e.g. $y_0=5, z_0=5, v_{y_0}=10, v_{z_0}=5, \theta_0=2, \omega_0=2, T=5$) to see how well the quadrotor drone returns to the origin.

In [ ]:
from importlib import reload
import additional.trajectory_simulation as trajectory_simulation
reload(trajectory_simulation)
from additional.trajectory_simulation import visualize_trajectory

visualize_trajectory(
    y0=5, z0=5, vy0=10, vz0=5, theta0=2, omega0=2, T=5,
    controller=controller
)

You have reached the end of Exercise 1, good job! We hope you enjoyed the learning journey and gained valuable insights. Keep experimenting, keep learning, and happy coding!

# Solutions:
Solutions for each problem are available, but it is highly encouraged to attempt solving them yourself first before revealing any solutuions. Solutions to all the problems can be found at the end of the notebook.

`NOTE`: There is no "one-solution" to any of these problems, and therefore your solution might not exactly match what we suggest. 

In [ ]:
# 1.1 VANILLA SOLUTION:
from torch import nn

# -------------------------------------------
#  TODO: Define the model architecture
# -------------------------------------------
model = nn.Sequential(
    nn.Linear(6, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 2),
    nn.Sigmoid()
)
# -------------------------------------------
# END
# -------------------------------------------

print(model)

In [ ]:
# 1.2 VANILLA SOLUTION:

def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    validate_every: int = 1,
):
    train_losses = []
    val_losses = []

    # -------------------------------------------
    #  TODO: Define the training loop
    # -------------------------------------------

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}")

        for inputs, targets in loop:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() # accumulate loss
            loop.set_postfix(train_loss=loss.item())    # progres bar update (current loss display)

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Optional validation
        if (epoch + 1) % validate_every == 0:
            val_loss = validate(model, val_loader)
            val_losses.append(val_loss)
            print(f"\nValidation Loss after Epoch {epoch + 1}: {val_loss:.4f}")

    # -------------------------------------------
    #  END
    # -------------------------------------------

    return train_losses, val_losses


def validate(model: nn.Module, val_loader: DataLoader) -> float:
    model.eval()
    total_loss = 0.0

    # -------------------------------------------
    #  TODO: Define the vlidation loop
    # -------------------------------------------

    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

    # -------------------------------------------
    # END
    # -------------------------------------------
    
    return total_loss / len(val_loader)


In [ ]:
# 1.3 VANILLA SOLUTION:
EPOCHS = 4
VALIDATE_EVERY = 2  # Every 2 epochs

# -------------------------------------------
# TODO: Complete training loop
# -------------------------------------------

train_losses, val_losses = train(
    model,
    train_loader,
    val_loader,
    optimizer,
    epochs=EPOCHS,
    validate_every=VALIDATE_EVERY,
)

# -------------------------------------------
# END
# -------------------------------------------

In [ ]:
# 1.4 VANILLA SOLUTION:

# Define controller
def controller(x: np.ndarray) -> np.ndarray:
    """
    Compute the control output using the trained model.

    Parameters:
        x (np.ndarray): The state vector.

    Returns:
        np.ndarray: Control output as a numpy array.
    """
    # -----------------------------------------------
    # TODO: Implement the controller function
    # -----------------------------------------------

    with torch.no_grad():
        u = model(torch.tensor(x, dtype=torch.float32))
    return u.numpy()

    # -----------------------------------------------
    # END
    # -----------------------------------------------


# Define system dynamics
from additional.system_dynamics import f